Prompt:
how to parse a text and then make ir code?

Output:
`pip install lark-parser llvmlite`

In [1]:
grammar = r"""
start: expr

?expr: expr "+" term -> add
     | expr "-" term -> sub
     | term

?term: term "*" factor -> mul
     | term "/" factor -> div
     | factor

?factor: NUMBER -> number
       | "(" expr ")"

%import common.NUMBER
%import common.WS
%ignore WS
"""


In [2]:
from lark import Lark, Transformer, v_args

expr_parser = Lark(grammar, parser="lalr")


In [8]:
import llvmlite.ir as ir

class ExprTransformer(Transformer):
    def __init__(self):
        self.builder = None
        self.module = ir.Module(name="expr_module")
        self.func = None
        self.create_main_function()

    def create_main_function(self):
        func_type = ir.FunctionType(ir.VoidType(), ())
        self.func = ir.Function(self.module, func_type, name="main")
        entry_block = self.func.append_basic_block(name="entry")
        self.builder = ir.IRBuilder(entry_block)

    @v_args(inline=True)
    def add(self, *children):
        return self.builder.fadd(children[0], children[1])

    @v_args(inline=True)
    def sub(self, *children):
        return self.builder.fsub(children[0], children[1])

    @v_args(inline=True)
    def mul(self, *children):
        return self.builder.fmul(children[0], children[1])

    @v_args(inline=True)
    def div(self, *children):
        return self.builder.fdiv(children[0], children[1])

    @v_args(inline=True)
    def number(self, token):
        return ir.Constant(ir.DoubleType(), float(token))

    def start(self, children):
        result = children[0]
        print("Result:", result)
        self.builder.ret_void()
        return self.module


In [9]:
input_expr = "3.0 * (2.0 + 4.0) / 2.0"

# Parse the input text
parse_tree = expr_parser.parse(input_expr)

# Generate LLVM IR
expr_transformer = ExprTransformer()
llvm_module = expr_transformer.transform(parse_tree)

print("Generated LLVM IR:")
print(llvm_module)


VisitError: Error trying to process rule "mul":

ExprTransformer.mul() takes 2 positional arguments but 3 were given

Prompt:
ExprTransformer.add() takes 2 positional arguments but 3 were given

In [5]:
import llvmlite.ir as ir

class ExprTransformer(Transformer):
    def __init__(self):
        self.builder = None
        self.module = ir.Module(name="expr_module")
        self.func = None
        self.create_main_function()

    def create_main_function(self):
        func_type = ir.FunctionType(ir.VoidType(), ())
        self.func = ir.Function(self.module, func_type, name="main")
        entry_block = self.func.append_basic_block(name="entry")
        self.builder = ir.IRBuilder(entry_block)

    def add(self, children):
        return self.builder.fadd(children[0], children[1])

    def sub(self, children):
        return self.builder.fsub(children[0], children[1])

    def mul(self, children):
        return self.builder.fmul(children[0], children[1])

    def div(self, children):
        return self.builder.fdiv(children[0], children[1])

    def number(self, token):
        return ir.Constant(ir.DoubleType(), float(token[0]))

    def start(self, children):
        result = children[0]
        print("Result:", result)
        self.builder.ret_void()
        return self.module


In [6]:
input_expr = "3.0 * (2.0 + 4.0) / 2.0"

# Parse the input text
parse_tree = expr_parser.parse(input_expr)

# Generate LLVM IR
expr_transformer = ExprTransformer()
llvm_module = expr_transformer.transform(parse_tree)

print("Generated LLVM IR:")
print(llvm_module)


Result: %".4" = fdiv double %".3", 0x4000000000000000
Generated LLVM IR:
; ModuleID = "expr_module"
target triple = "unknown-unknown-unknown"
target datalayout = ""

define void @"main"()
{
entry:
  %".2" = fadd double 0x4000000000000000, 0x4010000000000000
  %".3" = fmul double 0x4008000000000000, %".2"
  %".4" = fdiv double %".3", 0x4000000000000000
  ret void
}



In [7]:
import llvmlite.binding as llvmbinding

# Initialize LLVM
llvmbinding.initialize()
llvmbinding.initialize_native_target()
llvmbinding.initialize_native_asmprinter()

# Load the LLVM IR module
module = llvmbinding.parse_assembly(llvm_module)

# Create a JIT compiler
target_machine = llvmbinding.Target.from_default_triple().create_target_machine()
engine = llvmbinding.create_mcjit_compiler(module, target_machine)

# Compile and execute the `main` function
engine.finalize_object()
main_func = engine.get_function_address("main")
result = ctypes.CFUNCTYPE(ctypes.c_int32)(main_func)()

print("Result:", result)

AttributeError: 'Module' object has no attribute 'encode'

Prompt:
Can I make and axecutable